# Import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
import pandas as pd
import os
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

from agent import CardRecognizer
from utils.Loader import CardsDataset
from utils.evaluator import Evaluator

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
recognizer = CardRecognizer(csv_file="cards.csv", device="cpu")
dataset = CardsDataset(scale=0.6, split="test", csv_file="cards.csv", target="labels")

In [3]:
true_labels = []
pred_labels = []

for i in range(len(dataset)):
    image, true_label = dataset.__getitem__(i)
    category, suit = recognizer.classify_card(image)
    pred_label = f"{category} of {suit}"
    true_labels.append(str(dataset.decode_label(true_label)))
    pred_labels.append(pred_label)

df = pd.DataFrame({"True labels": true_labels, "Pred labels": pred_labels})

In [4]:
unique_labels = df["True labels"].unique().tolist()
df["Pred labels"] = df["Pred labels"].apply(lambda x: unique_labels.index(x) if x in unique_labels else -1)
df["True labels"] = df["True labels"].apply(lambda x: unique_labels.index(x) if x in unique_labels else -1)
df.head()

,True labels,Pred labels
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4


In [5]:
def plot_and_save_confusion_matrix(true_labels, pred_labels, save_path, num_parameters):
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Etiquetas Predichas")
    plt.ylabel("Etiquetas Verdaderas")
    plt.title(f"Matriz de Confusión. Num.Param{num_parameters}")
    plt.savefig(save_path)
    plt.close()
    
def evaluate_model(y_pred: np.array, y_test: np.array, class_names_str: list):
    df_result = pd.DataFrame({"Prediction": y_pred, "GroundTruth": y_test})
    df_result["Prediction"] = df_result["Prediction"].apply(lambda x: class_names_str[x])
    df_result["GroundTruth"] = df_result["GroundTruth"].apply(lambda x: class_names_str[x])
    return Evaluator.evaluate_classification_metrics(df_result)

In [6]:
plot_and_save_confusion_matrix(
    df["True labels"].tolist(),
    df["Pred labels"].tolist(),
    save_path="result/confusion_matrix.png",
    num_parameters=recognizer.size()
)

In [7]:
df_metrics = evaluate_model(df["Pred labels"], df["True labels"], range(len(unique_labels)))
df_metrics["Clase"] = df_metrics.index.to_series().iloc[0:len(unique_labels)].apply(lambda x: unique_labels[x])
df_metrics = df_metrics[["Clase"] + [col for col in df_metrics.columns if col != "Clase"]]
df_metrics.to_csv("result/metrics.csv", index=False)

# MobileNetV2

In [ ]:
from torchvision import models, transforms
from torch.utils.data import DataLoader

# Load MobileNet model
mobilenet = models.mobilenet_v2(pretrained=True)

# Modify the classifier for the number of classes in CardsDataset
num_classes = len(unique_labels)
mobilenet.classifier[1] = nn.Linear(mobilenet.last_channel, num_classes)

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Apply transformations to the dataset
# Create training and validation datasets
train_dataset = CardsDataset(scale=1, split="train", csv_file="cards.csv", target="labels", transform=transform, convert="RGB")
valid_dataset = CardsDataset(scale=1, split="valid", csv_file="cards.csv", target="labels", transform=transform, convert="RGB")

# Create dataloaders for training and validation
train_dataloader = DataLoader(train_dataset, batch_size=12, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=12, shuffle=False)

# Define optimizer and loss function
optimizer = optim.Adam(mobilenet.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Fine-tune the model
mobilenet.to(DEVICE)
mobilenet.train()
for epoch in range(100):  # Number of epochs
    running_loss = 0.0
    for images, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = mobilenet(images.to(DEVICE))
        loss = criterion(outputs, labels.to(DEVICE).float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    mobilenet.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in valid_dataloader:
            outputs = mobilenet(images.to(DEVICE))
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted.cpu() == labels).sum().item()
    val_precision = val_correct / val_total if val_total > 0 else 0
    mobilenet.train()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_dataloader), Validation Precision: {val_precision:.4f}")

/home/lingfeng/Desktop/venvs/pytorch/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lingfeng/Desktop/venvs/pytorch/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 4.253599231893366
Epoch 2, Loss: 2.859260244802995
Epoch 3, Loss: 2.226621383970434
Epoch 4, Loss: 1.7850872711701826
Epoch 5, Loss: 1.4614528851075606
Epoch 6, Loss: 1.1310780400579625
Epoch 7, Loss: 0.9140946946360848
Epoch 8, Loss: 0.8488961092450402
Epoch 9, Loss: 0.6017012745141983
Epoch 10, Loss: 0.4138714928518642
Epoch 11, Loss: 0.3737646028060805
Epoch 12, Loss: 0.35907745598392055
Epoch 13, Loss: 0.34076557951894676
Epoch 14, Loss: 0.23330926505679433
Epoch 15, Loss: 0.1803588472645391
Epoch 16, Loss: 0.22685672749172558
Epoch 17, Loss: 0.23562341132624584
Epoch 18, Loss: 0.21220411072400483
Epoch 19, Loss: 0.23216954618692398
Epoch 20, Loss: 0.14746089982376856
Epoch 21, Loss: 0.1614886601878838
Epoch 22, Loss: 0.07444561095061628
Epoch 23, Loss: 0.04635130122981288
Epoch 24, Loss: 0.03776202349796553
Epoch 25, Loss: 0.055351925886828794
Epoch 26, Loss: 0.042361335469071164
Epoch 27, Loss: 0.02219349231613292
Epoch 28, Loss: 0.02081285499628972


KeyboardInterrupt: 